In [1]:
import pandas as pd

# Data Set 확인

In [156]:
# def load_data(dataset_dir):
dataset_dir = "/opt/ml/dataset/train/train.csv"
pd_dataset = pd.read_csv(dataset_dir)
pd_dataset.head()

,id,sentence,subject_entity,object_entity,label,source
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,"{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...","{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...",no_relation,wikipedia
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,"{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...","{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...",no_relation,wikitree
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,"{'word': '광주FC', 'start_idx': 21, 'end_idx': 2...","{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx...",org:member_of,wikitree
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,"{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...","{'word': '박정부', 'start_idx': 22, 'end_idx': 24...",org:top_members/employees,wikitree
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,"{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_id...","{'word': '1967', 'start_idx': 0, 'end_idx': 3,...",no_relation,wikipedia


In [173]:
pd_dataset["subject_entity"].iloc[1927]

"{'word': '유나이티드 항공', 'start_idx': 0, 'end_idx': 7, 'type': 'ORG'}"

In [174]:
pd_dataset["object_entity"].iloc[1927]

"{'word': '스타 얼라이언스', 'start_idx': 72, 'end_idx': 79, 'type': 'ORG'}"

In [5]:
t = eval(pd_dataset["subject_entity"].iloc[0])
print(t)

{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26, 'type': 'ORG'}


In [6]:
pd_dataset.describe()

,id
count,32470.000000
mean,16234.500000
std,9373.425957
min,0.000000
25%,8117.250000
50%,16234.500000
75%,24351.750000
max,32469.000000


## 문장 내 subject-entity나 object_entity의 (word 위치가 잘못된 기록 경우)는 없었다.

In [162]:
for index in range(len(pd_dataset)):
    id_, sentence, sub_ent, obj_ent = pd_dataset[["id","sentence","subject_entity","object_entity"]].iloc[index]
    sub_word = eval(sub_ent)['word']
    sub_start = eval(sub_ent)['start_idx']
    sub_end = eval(sub_ent)['end_idx']
    obj_word = eval(obj_ent)['word']
    obj_start = eval(obj_ent)['start_idx']
    obj_end = eval(obj_ent)['end_idx']
    if sentence[sub_start:sub_end+1] != sub_word or sentence[obj_start:obj_end+1] != obj_word:
        print(id_)

## 중복 Data 확인

In [8]:
# 중복 데이터 확인
print(pd_dataset.nunique())

id                32470
sentence          28803
subject_entity    26340
object_entity     25704
label                30
source                3
dtype: int64


## subject_entity나 object_entity에 (단어가 두개 이상 들어간 경우)는 없었다.

In [9]:
# 혹시 subject나 object의 word에 단어가 두개인 것은 없는지 검사
sbj_cnt=sum([1 if not isinstance(word,str) else 0 for word in pd_dataset["subject_entity"].map(lambda x: eval(x)['word'])])
print('subject_entity word 이상치 데이터 수:', sbj_cnt)
obj_cnt=sum([1 if not isinstance(word,str) else 0 for word in pd_dataset["object_entity"].map(lambda x: eval(x)['word'])])
print('object_entity word 이상치 데이터 수:', obj_cnt)    

subject_entity word 이상치 데이터 수: 0
object_entity word 이상치 데이터 수: 0


## 학습 데이터 중 (중복되는 문장의 비율)이 약 21% 정도로 생각보다 많았다.

In [161]:
# 2개 이상 중복되는 문장들이 포함된 데이터 추출
dup_sentences = list(pd_dataset["sentence"].value_counts()[pd_dataset["sentence"].value_counts()>1].index)
dup_sent_dataset = pd_dataset[[True if sentence in dup_sentences else False for sentence in pd_dataset["sentence"]]]
print('총 데이터 수:', len(pd_dataset))
print('중복 문장 개수:', len(dup_sentences))
print('중복 문장이 포함된 데이터 개수:', len(dup_sent_dataset),'(',len(dup_sent_dataset)/len(pd_dataset)*100,'%)')
for id_ in dup_sent_dataset['id'].values:
    if id_ == 13496:
        print("ss")
dup_sent_dataset.head()

총 데이터 수: 32470
중복 문장 개수: 3423
중복 문장이 포함된 데이터 개수: 7090 ( 21.835540498922082 %)
ss


,id,sentence,subject_entity,object_entity,label,source
6,6,그에 따라 나폴리와 계약을 연장한 마라도나는 1989년 팀을 UEFA컵 정상으로 인...,"{'word': 'AC 밀란', 'start_idx': 64, 'end_idx': ...","{'word': '1989', 'start_idx': 25, 'end_idx': 2...",no_relation,wikipedia
7,7,"박용오(朴容旿, 1937년 4월 29일(음력 3월 19일)(음력 3월 19일) ~ ...","{'word': '박용오', 'start_idx': 0, 'end_idx': 2, ...","{'word': '1937년 4월 29일', 'start_idx': 9, 'end_...",per:date_of_birth,wikipedia
9,9,"특히 김동연 전 경제부총리를 비롯한 김두관 국회의원, 안규백 국회의원, 김종민 국회...","{'word': '안규백', 'start_idx': 30, 'end_idx': 32...","{'word': '더불어민주당', 'start_idx': 100, 'end_idx'...",per:employee_of,wikitree
10,10,하비에르 파스토레는 아르헨티나 클럽 타예레스의 유소년팀에서 축구를 시작하였다.,"{'word': '하비에르 파스토레', 'start_idx': 0, 'end_idx...","{'word': '아르헨티나', 'start_idx': 11, 'end_idx': ...",per:origin,wikipedia
12,12,"2009년 9월, 미국 프로 야구 필라델피아 필리스 소속의 야구 선수 박찬호는 《M...","{'word': '필라델피아 필리스', 'start_idx': 19, 'end_id...","{'word': '박찬호', 'start_idx': 39, 'end_idx': 41...",org:top_members/employees,wikipedia


## 중복 문장 내에서도 word위치에 따라 중의적 의미를 가질 수 있으므로 (같은 위치의 WORD만을 같은 단어라 가정)하고 중복문장을 찾아봤을떄
## => 총 97개의 문장이 중복됨(약 0.3% 수준)

## 또한 동일한 위치의 word 임에도 subject_entity와 object_entity의 위치가 바뀌었다고 type이 다른 경우도 있었음.
## => 총 16개(약 0.05% 수준)

In [67]:
from collections import defaultdict

tmp_dataset = defaultdict(dict) # {sentence_:{id:[] ,subject_entitiy:[] , object_entity:[], label:[]}

# print(dup_sent_dataset[["id","sentence","subject_entity","object_entity", "label"]].head())
# for s in dup_sent_dataset[["id","sentence","subject_entity","object_entity", "label"]].head():
#     print(s)
# len(dup_sent_dataset)

# print(id_, sentence, sub, obj, label)

for index in range(len(dup_sent_dataset)):
    id_, sentence, sub, obj, label = dup_sent_dataset[["id","sentence","subject_entity","object_entity", "label"]].iloc[index]
    # 중복되는 문장의 처음 자료는 그대로 저장 후 다음 문장으로 넘어감
    # odd_id: word는 일치하는데 type 태깅이 잘못된 데이터
    # dup_id: sub,obj의 종류와 word,type까지 완전히 일치하는 데이터
    if len(tmp_dataset[sentence].keys()) == 0:
        tmp_dataset[sentence] = {"id":[id_], "sub":[sub], "obj":[obj], "label":[label], "odd_id":[], "dup_id":[]}
        continue
    
    # 이전 중복 데이터와 비교
    
    # 기존 중복문장의 정보 받아옴(리스트)
    o_sub, o_obj = tmp_dataset[sentence]["sub"], tmp_dataset[sentence]["obj"] # 리스트
    
    # 현재 문장의 정보 받아옴(딕셔너리)
    t_sub, t_obj = eval(sub), eval(obj)
    t_sub_word, t_sub_type = t_sub['word']+str(t_sub['start_idx'])+str(t_sub['end_idx']), t_sub['type']
    t_obj_word, t_obj_type = t_obj['word']+str(t_obj['start_idx'])+str(t_obj['end_idx']), t_obj['type']
    
    # 기존 중복문장과 현재 문장비교
    case = 0 # 0: 아무관련 없음, 1: 한 단어만 일치, 2: 2단어가 일치, 3: type태깅 오류
    for i in range(len(o_sub)): # 중복 별로 문장 순회
        o_sub_word, o_sub_type = eval(o_sub[i])['word']+str(eval(o_sub[i])['start_idx'])+str(eval(o_sub[i])['end_idx']), eval(o_sub[i])['type']
        o_obj_word, o_obj_type = eval(o_obj[i])['word']+str(eval(o_obj[i])['start_idx'])+str(eval(o_obj[i])['end_idx']), eval(o_obj[i])['type']
        case = 0
        # 두 문장간 연관성 체크
        for o_entity, o_word, o_type in [['sub',o_sub_word,o_sub_type], ['obj',o_obj_word,o_obj_type]]:
            for t_entity, t_word, t_type in [['sub',t_sub_word,t_sub_type], ['obj',t_obj_word,t_obj_type]]:
                if o_word == t_word:
                    if o_type == t_type and o_entity == t_entity:
                        case += 1
                    elif o_type != t_type:
                        case = 3
                if case >= 2:
                    break
            if case >= 2:
                break
        if case >= 2:
            break
            
    if case == 2: # sub,obj까지 완전 중복 일치
        tmp_dataset[sentence]["dup_id"].append(id_)
    elif case == 3:
        tmp_dataset[sentence]["odd_id"].append(id_)
    else:
        tmp_dataset[sentence]["id"].append(id_)
        tmp_dataset[sentence]["sub"].append(sub)
        tmp_dataset[sentence]["obj"].append(obj)
        tmp_dataset[sentence]["label"].append(label)


In [145]:
# 중복 데이터
final_dup_id = []
final_dup_id_del = []

for sent_, dic_ in tmp_dataset.items():
    if len(tmp_dataset[sent_]["dup_id"]) > 0:
        final_dup_id.extend(tmp_dataset[sent_]["id"])
        final_dup_id.extend(tmp_dataset[sent_]["dup_id"])
        final_dup_id_del.extend(tmp_dataset[sent_]["dup_id"])
    
# 태깅오류 데이터
final_odd_id = []

for sent_, dic_ in tmp_dataset.items():
    if len(tmp_dataset[sent_]["odd_id"]) >0:
        final_odd_id.extend(tmp_dataset[sent_]["id"])
        final_odd_id.extend(tmp_dataset[sent_]["odd_id"])


print('='*30+"중복 데이터"+'='*30)
print('총 중복 데이터 수:', len(final_dup_id),'(',len(final_dup_id)/len(pd_dataset)*100,'%)')
s = ''
for id_ in final_dup_id:
    if s != pd_dataset.iloc[id_]['sentence']:
        s = pd_dataset.iloc[id_]['sentence']
        print(s)
    print('id:', pd_dataset.iloc[id_]['id'])
    print('             sub_entity:', pd_dataset.iloc[id_]['subject_entity'])
    print('             obj_entity:', pd_dataset.iloc[id_]['object_entity'])
    print('                  label:', pd_dataset.iloc[id_]['label'])

print('='*30+"태깅오류 데이터 샘플"+'='*30)
print("태깅오류 데이터 수:", len(final_odd_id),'(',len(final_odd_id)/len(pd_dataset)*100,'%)')
s = ''
for id_ in final_odd_id:
    if s != pd_dataset.iloc[id_]['sentence']:
        s = pd_dataset.iloc[id_]['sentence']
        print(s)
    print('id:',pd_dataset.iloc[id_]['id'])
    print('             sub_entity:', pd_dataset.iloc[id_]['subject_entity'])
    print('             obj_entity:', pd_dataset.iloc[id_]['object_entity'])
    print('                  label:', pd_dataset.iloc[id_]['label'])

==============================중복 데이터==============================
총 중복 데이터 수: 97 ( 0.29873729596550663 %)
2010년 10월 15일 전라북도청을 방문한 맹형규 행정안전부 장관은 도내 지자체간 미묘한 갈등을 보이고 있는 새만금 행정구역 개편에 대해 "매우 골치 아픈 일이지만 새만금 내부개발에 있어 중요한 일인 만큼 분쟁이 비교적 적은 3~4호 방조제 구간에 대해서는 제5차 중앙분쟁조정위원회에서 결론을 낼 것으로 본다"며 "나머지 구간에 대해서도 지자체간의 협의점을 도출해 내 조속히 마무리될 수 있도록 노력하겠다"고 말했다.
id: 255
             sub_entity: {'word': '전라북도청', 'start_idx': 14, 'end_idx': 18, 'type': 'ORG'}
             obj_entity: {'word': '행정안전부', 'start_idx': 29, 'end_idx': 33, 'type': 'ORG'}
                  label: no_relation
id: 3547
             sub_entity: {'word': '전라북도청', 'start_idx': 14, 'end_idx': 18, 'type': 'ORG'}
             obj_entity: {'word': '행정안전부', 'start_idx': 29, 'end_idx': 33, 'type': 'ORG'}
                  label: no_relation
이날 프로그램 공개에서는 전북영산작법보존회와 김명신‧정상희의 사제동행 판소리, 관악 명인 대금 원장현, 색소폰 강태환의 무대를 선보이며 올해 축제에 대한 기대감을 한껏 끌어 올렸다.
id: 277
             sub_entity: {'word': '강태환', 'start_idx': 62, 'end_idx': 64, 'type': 'PER'}
         